In [6]:
# !apt-get install p7zip
# !p7zip -d -f -k ./mercari-price-suggestion-challenge/train.tsv.7z
# !unzip -o ./mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
# !unzip -o ./mercari-price-suggestion-challenge/test_stg2.tsv.zip
# !p7zip -d -f -k ./mercari-price-suggestion-challenge/test.tsv.7z
# !pip install pandas
# !pip install sklearn
# !pip install h2o
# !git clone https://github.com/4paradigm/autox.git
# !pip install ./autox
# !pip install psutil


In [7]:
import numpy as np 
import pandas as pd 
# from nltk.corpus import stopwords
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import LabelBinarizer
# from scipy.sparse import hstack, csr_matrix
# from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
import re
import gc
import matplotlib.pyplot as plt
#H2O
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time
import os
import psutil

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [13]:
train = pd.read_csv("sub_train.csv")
test1 = pd.read_csv('sub_val.csv')

In [14]:
test = test1.drop(columns=['target'])

In [15]:
test.head(3)

,id,url_legal,license,excerpt,standard_error
0,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as quickly as it came, and two days after their sleigh ride there was scarcely a vestige of white on the ground. Tennis was again possible and a great game was in progress on the court at Pine Laurel. Patty and Roger were playing against Elise and Sam Blaney, and the pairs were well matched.\nBut the long-contested victory finally went against Patty, and she laughingly accepted defeat.\n""Only because Patty's not quite back on her game yet,"" Roger defended; ""this child has been on the sick list, you know, Sam, and she isn't up to her own mark.""\n""Well, I like that!"" cried Patty; ""suppose you bear half the blame, Roger. You see, Mr. Blaney, he is so absorbed in his own Love Game, he can't play with his old-time skill.""\n""All right, Patsy, let it go at that. And it's so, too. I suddenly remembered something Mona told me to tell you, and it affected my service.""",0.47668
1,dd1000b26,NaN,NaN,"And outside before the palace a great garden was walled round, filled full of stately fruit-trees, gray olives and sweet figs, and pomegranates, pears, and apples, which bore the whole year round. For the rich south-west wind fed them, till pear grew ripe on pear, fig on fig, and grape on grape, all the winter and the spring. And at the farther end gay flower-beds bloomed through all seasons of the year; and two fair fountains rose, and ran, one through the garden grounds, and one beneath the palace gate, to water all the town. Such noble gifts the heavens had given to Alcinous the wise.\nSo they went in, and saw him sitting, like Poseidon, on his throne, with his golden sceptre by him, in garments stiff with gold, and in his hand a sculptured goblet, as he pledged the merchant kings; and beside him stood Arete, his wise and lovely queen, and leaned against a pillar as she spun her golden threads.",0.45001
2,37c1b32fb,NaN,NaN,"Once upon a time there were Three Bears who lived together in a house of their own in a wood. One of them was a Little, Small, Wee Bear; and one was a Middle-sized Bear, and the other was a Great, Huge Bear. They had each a pot for their porridge; a little pot for the Little, Small, Wee Bear; and a middle-sized pot for the Middle Bear; and a great pot for the Great, Huge Bear. And they had each a chair to sit in; a little chair for the Little, Small, Wee Bear; and a middle-sized chair for the Middle Bear; and a great chair for the Great, Huge Bear. And they had each a bed to sleep in; a little bed for the Little, Small, Wee Bear; and a middle-sized bed for the Middle Bear; and a great bed for the Great, Huge Bear.",0.51084


In [2]:
# train = pd.read_csv('train.csv')
# kf = KFold(n_splits=3, random_state=1001,shuffle=True)
# for i, (train_index, val_index) in enumerate(kf.split(train)):
#     trn= train.iloc[train_index].reset_index()
#     val= train.iloc[val_index].reset_index()
# val = val.drop(columns=['index'])
# test = val.drop(columns=['price'])
# test = test.rename(columns={'train_id':'test_id'})
# val = val.rename(columns={'train_id':'test_id'})
# test.to_csv('test.csv',index=False)
# val.to_csv('val.csv',index=False)
# train.to_csv('train.csv',index=False)
# !source /etc/profile
# !java

In [16]:
h2o.init()
all_train = h2o.H2OFrame(train)
# all_test = h2o.H2OFrame(test1)
#Get columns names for Building H2O Models
target = 'target'
predictors = [f for f in all_train.columns if f not in ['target']]

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17" 2021-09-14; OpenJDK Runtime Environment (build 17+35-2724); OpenJDK 64-Bit Server VM (build 17+35-2724, mixed mode, sharing)
  Starting server from /root/miniconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpjsfdryem
  JVM stdout: /tmp/tmpjsfdryem/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpjsfdryem/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,21 days and 23 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_8240p8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20 Gb
H2O_cluster_total_cores:,15
H2O_cluster_allowed_cores:,15
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [17]:
from h2o.automl import H2OAutoML

In [18]:
aml = H2OAutoML(max_models=1, seed=1331)
aml.train(x=predictors, y=target, training_frame=all_train)

AutoML progress: |
15:55:31.597: _train param, Dropping bad and constant columns: [excerpt, id]

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220504_155531


Model Summary: 


,,number_of_trees
0,,30.00000




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.7744582581118759
RMSE: 0.880033100577402
MAE: 0.684357077229975
RMSLE: NaN
Mean Residual Deviance: 0.7744582581118759

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 1.0233189592749248
RMSE: 1.0115922890546984
MAE: 0.7982298570490447
RMSLE: NaN
Mean Residual Deviance: 1.0233189592749248

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.79823,0.02735,0.84141,0.76651,0.78741,0.79918,0.79666
1,mean_residual_deviance,1.02332,0.05207,1.07592,0.93663,1.02417,1.03793,1.04195
2,mse,1.02332,0.05207,1.07592,0.93663,1.02417,1.03793,1.04195
3,r2,0.02699,0.04807,0.07139,0.02489,0.07905,-0.01293,-0.02745
4,residual_deviance,1.02332,0.05207,1.07592,0.93663,1.02417,1.03793,1.04195
5,rmse,1.01132,0.02604,1.03727,0.96780,1.01201,1.01879,1.02076
6,rmsle,NaN,0.00000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-04 15:55:33,2.334 sec,0.00000,1.79899,1.52707,3.23636
1,,2022-05-04 15:55:33,2.371 sec,5.00000,0.97112,0.78189,0.94307
2,,2022-05-04 15:55:33,2.396 sec,10.00000,0.92686,0.73974,0.85908
3,,2022-05-04 15:55:34,2.608 sec,15.00000,0.90962,0.71611,0.82740
4,,2022-05-04 15:55:34,2.655 sec,20.00000,0.90057,0.70298,0.81102
5,,2022-05-04 15:55:34,2.689 sec,25.00000,0.88753,0.69186,0.78772
6,,2022-05-04 15:55:34,2.743 sec,30.00000,0.88003,0.68436,0.77446



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,standard_error,1044.45007,1.00000,0.74896
1,license.CC BY 4.0,191.46297,0.18331,0.13730
2,url_legal.nan,51.94466,0.04973,0.03725
3,license.nan,33.43918,0.03202,0.02398
4,license.CC BY-SA 3.0,30.96078,0.02964,0.02220
5,license.CC BY-SA 3.0 and GFDL,20.85509,0.01997,0.01495
6,url_legal.https://www.africanstorybook.org/,11.76500,0.01126,0.00844
7,url_legal.https://www.africanstorybook.org/#,7.62610,0.00730,0.00547
8,license.CC BY-NC-SA 2.0,2.02161,0.00194,0.00145


In [19]:
test = h2o.H2OFrame(test)
prediction = aml.leader.predict(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/root/miniconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'url_legal' has levels not trained on: ["http://static.ehe.osu.edu/sites/beyond/penguins/downloads/feature-stories/erebus-23-text.pdf", "http://static.ehe.osu.edu/sites/beyond/penguins/downloads/feature-stories/getting-warmer-k1-text.pdf", "http://static.ehe.osu.edu/sites/beyond/penguins/downloads/feature-stories/partners-23-text.pdf", "http://static.ehe.osu.edu/sites/beyond/penguins/downloads/feature-stories/tundra-k1-text.pdf", "https://drive.google.com/file/d/0B6pcGxi_-Y9PRzdONUY2a1Y5aTg/view", "https://drive.google.com/file/d/0Bz69EszGvRvaYnlFU1Zfb0hHNXc/view", "https://emedia.uen.org/courses/utah-oer-textbooks-3rd-grade-seed/view", "https://en.wikipedia.org/wiki/Abyssal_plain", "https://en.wikipedia.org/wiki/Active_noise_control", "https://en.wikipedia.org/wiki/Aerodynamics", ...200 not listed..., "https://www.commonlit.org/texts/the-sit-in-movement", "https://www.commonlit.org/

In [20]:
prediction[['predict']]

predict
-0.889398
-1.08929
-1.68389
-0.530998
-1.12602
-1.03916
-0.860144
-0.972443
-1.71364
-1.3932


In [21]:
pred = prediction.as_data_frame()

In [22]:
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(test1['target'], pred))

In [23]:
RMSE

0.9980093661065355

In [3]:
import time
import os
import psutil
import numpy as np 
import pandas as pd 
from collections import OrderedDict

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info
import h2o
from h2o.estimators import H2OWord2vecEstimator, H2OGradientBoostingEstimator
df_train = pd.read_csv('sub_train.csv')
documents = df_train['excerpt'].to_list()
doc_ids = list(range(len(documents)))
@count_time
@count_info
def calcu_h2o():
    h2o.init()
    input_frame = h2o.H2OFrame(OrderedDict([('DocID', doc_ids), ('Document', documents)]),
                                    column_types=['numeric', 'string'])
    STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can",
                  "lines","re","what","there","all","we","one","the",
                  "a","an","of","or","in","for","by","on","but","is",
                  "in","a","not","with","as","was","if","they","are",
                  "this","and","it","have","from","at","my","be","by",
                  "not","that","to","from","com","org","like","likes",
                  "so"]
    # Make the 'tokenize' function:
    def tokenize(sentences, stop_word = STOP_WORDS):
        tokenized = sentences.tokenize("\\W+")
        tokenized_lower = tokenized.tolower()
        tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
        tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
        tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
        return tokenized_words

    words = tokenize(input_frame["Document"])

    # Build word2vec model:
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
    w2v_model.train(training_frame=words)

    # Find synonyms for the words "teacher":
    w2v_model.find_synonyms("teacher", count = 5)

    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [4]:
calcu_h2o()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 11 mins
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_unknownUser_ftgn45
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,19.95 Gb
H2O_cluster_total_cores:,15
H2O_cluster_allowed_cores:,15
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
程序占用了内存3684.0KB
程序运行了32.32680058479309秒
